In [167]:
import numpy as np

# Imagine you have a class with a .update method, 
# args are the pose of an object and 
# your .update method gets called every time you have a new pose estimate (about 100hz)

# https://cseweb.ucsd.edu/classes/wi18/cse169-a/slides/CSE169_03.pdf

# 3d position vector
# 4x4 pose matrix: review this, as well as homogeneous coordinates.
# https://peteroupc.github.io/html3dutil/tutorial-matrixdetails.html#Why_a_4x4_Matrix

def hamilton(q1, q2):
        q_n = np.zeros(4)
        q_n[0] = q1[0]*q2[0] - q1[1]*q2[1] - q1[2]*q2[2] - q1[3]*q2[3]
        q_n[1] = q1[0]*q2[1] + q1[1]*q2[0] + q1[2]*q2[3] - q1[3]*q2[2]
        q_n[2] = q1[0]*q2[2] - q1[1]*q2[3] + q1[2]*q2[0] + q1[3]*q2[1]
        q_n[3] = q1[0]*q2[3] + q1[1]*q2[2] - q1[2]*q2[1] + q1[3]*q2[0]
        return q_n
    
def inverse(q):
    mag = q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2 # should just be 1 bc unit quaternion.
    return mag * np.array([q[0], -q[1], -q[2], -q[3]])

class Object: # stored pose is the previous pose

    def __init__(self, stored_pose_mat, stored_pose_quat):
        self.stored_pose_mat = stored_pose_mat 
        self.stored_pose_quat = stored_pose_quat
    
    ######## In quat space ###########
    
    def angle_between_quats(self, q1, q2): # we are assuming the paramter quat 'q' is already normalized
        return 2*np.arccos(np.clip(q1@q2,a_min=-1,a_max=1))

    def axis_between_quats(self, q1, q2):
        ## finish this: rotational axis for q1 -> q2 rotation
        import pdb; pdb.set_trace()
        q_rel = hamilton(inverse(q1), q2)
        theta = 2*np.arccos(np.clip(q_rel[0],a_min=-1,a_max=1))
        versor = q_rel[1:4]
        norm = np.linalg.norm(versor)
        if (norm != 0):
            versor = versor / np.linalg.norm(versor, 2)
        axis = versor
        return axis
    
    def update_quat(self, cur_quat):
        # import pdb; pdb.set_trace()
        theta = self.angle_between_quats(self.stored_pose_quat, cur_quat)
        axis = self.axis_between_quats(self.stored_pose_quat, cur_quat)
        self.stored_pose_quat = cur_quat
        return [theta, axis]

    ######## In rot space #############

    def angle_between_rotations(self, R1, R2):
        Rdiff = R2@np.transpose(R1) # rotation matrix required to obtain R2 from R1 (R2 = Rdiff@R1, Rdiff = R255)
        trace = np.trace(Rdiff)
        theta = np.arccos((trace - 1)/2)
        return theta
    
    def axis_between_rotations(self, R1, R2):
        # find the eigenvector, where the eigenvalue=1
        # import pdb; pdb.set_trace()
        Rdiff = np.transpose(R1)@R2
        eigs = np.linalg.eig(Rdiff)
        eigenval1 = 1 + 0j
        ind = np.where(eigs[0] == eigenval1) # identify index of rotation axis eigenvector (vector whose eigenvalue is 1)
        axis = eigs[1][:,ind[0].item()]
        return axis
    
    # will get called every time a new pose estimate arrives (~100 times/sec)
    def update_mat(self, cur_pose):
        # compute angular velocity of the object
        # so we probably need to compare the orientation of the current time step, with the previous one
        # and find the angular velocity with respect to a 3d-axis
        theta = self.angle_between_rotations(self.stored_pose_mat, cur_pose)
        axis = self.axis_between_rotations(self.stored_pose_mat, cur_pose)
        self.stored_pose_mat = cur_pose # update the stored pose matrix
        return [theta, axis] # return both as a tuple
    


## Then, do the same, but implemented efficiently in C++
    

In [168]:
# Define Rotation matrices as a proxy for Transformation pose matrices. We only need to calculate angular velocity, so no translation information is required.
# !! printing incorrect axis, check the hamilton product function.

rad45 = 45 * np.pi/180
rad20 = 20 * np.pi/180
# +45 degree rotation about the x-axis
Rx45 = np.array([[1, 0, 0], [0, np.cos(rad45), -np.sin(rad45)], [0, np.sin(rad45), np.cos(rad45)]])
# +20 degree rotation about the x-axis
Rx20 = np.array([[1, 0, 0], [0, np.cos(rad20), -np.sin(rad20)], [0, np.sin(rad20), np.cos(rad20)]])

q45x = np.array([ 0.3826834, 0, 0, 0.9238795]) # orientation: +45 degrees about x-axis
q20x = np.array([ 0.1736482, 0, 0, 0.9848078]) # orientation: +20 degrees about x-axis\

o1 = Object(Rx45, q45x)
[theta, axis] = o1.update_mat(Rx20)
# print(o1.update_mat(Rx20))
print(f'object experience a: {theta*180/np.pi} degree rotation, about the axis {np.real(axis)}')
print(f'assuming 100 Hz, the angular velocity is {theta*100} rad/sec')

object experience a: 24.999999999999993 degree rotation, about the axis [1. 0. 0.]
assuming 100 Hz, the angular velocity is 43.633231299858224 rad/sec


In [ ]:
# Perform the same operations with quaternions:

[theta, axis] = o1.update_quat(q20x)
# print(o1.update_mat(Rx20))
print(f'object experience a: {theta*180/np.pi} degree rotation, about the axis {np.real(axis)}')
print(f'assuming 100 Hz, the angular velocity is {theta*100} rad/sec')


> c:\users\joaqu\appdata\local\temp\ipykernel_17460\2326154756.py(39)axis_between_quats()



ipdb>  n


> c:\users\joaqu\appdata\local\temp\ipykernel_17460\2326154756.py(40)axis_between_quats()



ipdb>  n


> c:\users\joaqu\appdata\local\temp\ipykernel_17460\2326154756.py(41)axis_between_quats()



...

array([0.97629594, 0.        , 0.        , 0.21643957])


ipdb>  inverse(q1)


array([ 0.38268337, -0.        , -0.        , -0.92387942])


ipdb>  q2


array([0.1736482, 0.       , 0.       , 0.9848078])


In [29]:




# pose 1 is rotated 45 degrees about the x-axis, and located at [1,1,1].
# pose 2 is rotated 20 degrees about the x-axis, and located at [2,2,1].
pose1 = [[1,  0,  0, 1],
   [0,  1, 0, 1],
   [0,  0,  1,  1],
   [0, 0, 0, 1]]

pose2 = [[1,  0,  0, 2],
   [0,  0.9396926, -0.3420202, 2],
   [0,  0.3420202,  0.9396926, 2],
   [0, 0, 0, 1]]

o1 = Object(pose1)
angular_vel = o1.update(pose2)

print(f'angular velocity is: {angular_vel} radians per second')


v2 is: [0 0 1 0]
v3 is: [ 0.        -0.3420202  0.9396926  0.       ]
angular velocity is: 93.96926001396383 radians per second


In [11]:



### Filter questions: ###
# (discrete exponential filters to filter noise in the incoming signal) #






## Then, do the same, but implemented efficiently in c++

In [65]:
rad45 = 45 * np.pi/180
rad20 = 20 * np.pi/180

Rx45 = np.array([[1, 0, 0], [0, np.cos(rad45), -np.sin(rad45)], [0, np.sin(rad45), np.cos(rad45)]])
Rx20 = np.array([[1, 0, 0], [0, np.cos(rad20), -np.sin(rad20)], [0, np.sin(rad20), np.cos(rad20)]])

theta = angle_between_rotation_matrices(Rx45,Rx20)
print(f'theta is: {theta*180/np.pi} degrees')

eigs = np.linalg.eig(np.transpose(Rx45)@Rx20)
eigenval1 = 1 + 0j
ind = np.where(eigs[0] == eigenval1)
print(eigs[0][ind])

theta is: 24.999999999999993 degrees
[1.+0.j]


In [48]:
### quaternion version: ###

def angle_between_quaternions(q1, q2): # we are assuming the paramter quat 'q' is already normalized
    return 2*np.arccos(np.clip(q1@q2,a_min=-1,a_max=1))

q1 = np.array([ 0.3826834, 0, 0, 0.9238795]) # orientation: +45 degrees about x-axis
q2 = np.array([ 0.1736482, 0, 0, 0.9848078]) # orientation: +20 degrees about x-axis\
# i.e., we should see a 25 degree difference

print(f'angle between quaternions: {angle_between_quaternions(q1,q2) * 180 / np.pi} degrees')

angle between quaternions: 24.999992418408635 degrees
